In [47]:
import sqlite3
import json
from datetime import datetime
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.layers import Embedding
from keras.layers import Flatten
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb
from keras import preprocessing
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import os
from spacy.lang.en import English
from keras.layers import SimpleRNN
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras import utils
import re
import itertools
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model

# Importing data from a medical Q&A dataset containing over 135,000 questions and answer pairs

In [48]:
with open("healthtapQAs.json") as f: # This is the biggest database - 137,052 questions and answers
    my_data1 = json.load(f)

In [49]:
the_answers = list()
the_questions = list()

count = 0
count1 = 0

for i in my_data1:
    the_answers.append(i['answer'])
    count += 1

for i in my_data1:
    the_questions.append(i['question'])
    count1 += 1
    
print(the_questions[0])
print(the_answers[0])

zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.
a majority of the dental implants placed are titanium. they are highly successful with many years use ; many studies much lower in cost ; have many restorative options. zirconia implants are newer fewer studies on success lesser restorative options. however they can be more aesthetic in certain anterior(front) situations. let your dentist/oral surgeon chose what they feel will be best for you.


# Cleaning the data to get rid of some contractions 
## In the future I am thinking about applying this to each question written by a user

In [50]:
def clean_text(text):

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [51]:
answers = list()
questions = list()
for i in the_answers:
    text = clean_text(i)
    answers.append(text)

for i in the_questions:
    text = clean_text(i)
    questions.append(text)
    
print(the_questions[:4])
print(questions[:4])

['zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.', 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.', 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.', 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.']
['zirconium dental implants how common is it used now is there any advantages or benefits over titanium implants cons & pros please thanks', 'zirconium dental implants how common is it used now is there any advantages or benefits over titanium implants cons & pros please thanks', 'zirconium dental implants how common is it used now is there any advantages or benefits over titanium implants cons & pros please thanks'

# Adding 'START' and 'END' to the beginnings and endings of the answers

In [52]:
print(answers[1:4])
labeled_answers = list()
for i in range(len(answers)):
    labeled_answers.append('<START>' + answers[i] + '<END>')


['and the data on zirconia implants is much more limited', 'dental implants when loaded transfer stress to the bone which is a good thing zirconium is a ceramic that is very hard and actually transfers less stress to the bone and can actually lead to bone atrophy  the you dont use it you lose it concept', 'stick with what we know works  titanium let the research continue with the zirconium more answers will come as more research is done hard to beat 50 years of success always looking for improvement though']


# Tokenizing words in both the questions and answers list

In [53]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(questions + labeled_answers)
print("Vocab list has: {} elements".format(len(tokenizer.word_index)))
word_index = tokenizer.word_index

Vocab list has: 87816 elements


# Preprocessing Questions Data
## - Tokenize words
## - Create word integer index
## - Turn text to number sequence and padding

In [54]:
sequences_questions = tokenizer.texts_to_sequences(questions)
max_seq_len = max([len(x) for x in sequences_questions])
print("Longest sequence_questions is: {}".format(max_seq_len))
padded_questions = pad_sequences(sequences_questions, maxlen=max_seq_len)
# array of padded sequences for questions
encoder_input_data = np.array(padded_questions) 

print("Sequences are of dimensions: {}".format(encoder_input_data.shape))
print("Example sequence: {}".format(encoder_input_data[0]))
print("Example of WordIndex: {}".format(dict(itertools.islice(word_index.items(), 4))))

Longest sequence_questions is: 64
Sequences are of dimensions: (137052, 64)
Example sequence: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0 15221  1308   201    45   140     4
    11   155   136     4    38    65  3576    21   830   128  8161   201
  2943  3163   149  1030]
Example of WordIndex: {'the': 1, 'to': 2, 'a': 3, 'is': 4}


# Preprocessing Answers Data

In [55]:
sequences_answers = tokenizer.texts_to_sequences(labeled_answers)
max_seq_lenAns = 100
# max_seq_lenAns = max([len(x) for x in sequences_answers])
print("Longest sequence_answers is: {}".format(max_seq_lenAns))
padded_answers = pad_sequences(sequences_answers, maxlen=max_seq_lenAns)
# array of padded sequences for questions
decoder_input_data = np.array(padded_answers) 

print("Sequences are of dimensions: {}".format(decoder_input_data.shape))
print("Example sequence: {}".format(encoder_input_data[0]))
print("Example of WordIndex: {}".format(dict(itertools.islice(word_index.items(), 4))))

Longest sequence_answers is: 100
Sequences are of dimensions: (137052, 100)
Example sequence: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0 15221  1308   201    45   140     4
    11   155   136     4    38    65  3576    21   830   128  8161   201
  2943  3163   149  1030]
Example of WordIndex: {'the': 1, 'to': 2, 'a': 3, 'is': 4}


# Decoder output data

In [56]:
#experiment with the output dim 
output_dim = 200
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(max_words, output_dim, mask_zero=True) (encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(output_dim, return_state=True) (encoder_embedding)
# encoder_states = state_h, state_c


decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(max_words, output_dim, mask_zero=True) (decoder_inputs)
decoder_LSTM = LSTM(output_dim, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(max_words, activation='softmax')
output = decoder_dense (decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], output)

In [57]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [58]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 200)    4000000     input_7[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 200)    4000000     input_8[0][0]                    
____________________________________________________________________________________________

# Removing the word START from the answer sequences and padding the sequences

In [59]:
decoder_output_seq = sequences_answers
for i in range(len(decoder_output_seq)):
    decoder_output_seq[i] = decoder_output_seq[i][1:]

target_data = pad_sequences(decoder_output_seq, max_seq_lenAns)


# Shuffling then splitting the input data and target data 80:20 

In [60]:
# should all share the shape (137052, _)
print("Encoder Input: ", encoder_input_data.shape)
print("Decoder Input: ", decoder_input_data.shape)
print("Target Data Input: ", target_data.shape)

Encoder Input:  (137052, 64)
Decoder Input:  (137052, 100)
Target Data Input:  (137052, 100)


In [61]:
eighty = 0.8 * encoder_input_data.shape[0]
print(eighty)
train_perc = 109500/encoder_input_data.shape[0]
print(train_perc)

109641.6
0.7989668155152789


In [62]:
# Since I will be fitting the model in portion sizes of 300 I am going
# use 109,500 which is approximately 80% (actually 79.897%)

In [63]:
indices = np.arange(encoder_input_data.shape[0])
np.random.shuffle(indices) 
# EID = encoder_input_data
shuffled_EID = encoder_input_data[indices]
shuffled_DID = decoder_input_data[indices]
shuffled_target = target_data[indices]

print(shuffled_EID.shape)
print(shuffled_DID.shape)
print(shuffled_target.shape)

(137052, 64)
(137052, 100)
(137052, 100)


In [64]:
train_encoder_data = shuffled_EID[0:12000] 
train_decoder_data = shuffled_DID[0:12000]
train_target_data = shuffled_target[0:12000]

test_encoder_data = shuffled_EID[12000: 15001 ] 
test_decoder_data = shuffled_DID[12000: 15001]
test_target_data = shuffled_target[12000: 15001]

print(train_encoder_data.shape)
print(train_decoder_data.shape)
print(train_target_data.shape)
print(test_encoder_data.shape)
print(test_decoder_data.shape)
print(test_target_data.shape)

print(train_encoder_data[1])
print(train_decoder_data[1])
print(train_target_data[1])

print(int(train_target_data.shape[0]/300))

(12000, 64)
(12000, 100)
(12000, 100)
(3001, 64)
(3001, 100)
(3001, 100)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0   27   56  812  113    2  253  151    1 4694
 7179  427   61    3  509   24  596    6  129  204   16   38    5   35
   70   15  231   11    4  105  267  185]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    7 2672   41  532  540  933   18   44
   58  191   77 1082   25   86   21    3 1714   12   39   81 8231  247
   11    4 6327  834   92 5643 1509   11    9  403   19  151    3  994
  449  161    5   10   46   15 4328   22    9   17  296  204   41 1509
   11    8]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0 

# Fitting the model in portions since the OneHotEncoded Answers are too large to be held in computer memory

In [65]:
tenth = 300
times_completed = int(train_target_data.shape[0]/300)
for i in range(times_completed): # CHANGE THIS
    current = i * tenth
    nextone = (i+1) * tenth
    print("Currently--> {}:{}".format(current, nextone))
    train_X1 = train_encoder_data[current:nextone]
    train_X2 = train_decoder_data[current:nextone]
    portion = train_target_data[current:nextone]
    output = utils.to_categorical(portion, max_words)
#     print(train_X1.shape)
#     print(train_X2.shape)
#     print(output.shape)
#     print(output)
    model.fit([train_X1 , train_X2], output, batch_size=32, epochs=20 ) 

Currently--> 0:300
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
300/300 [==============================] - ETA: 1:03 - loss: 9.903 - ETA: 44s - loss: 9.900 - ETA: 35s - loss: 9.89 - ETA: 28s - loss: 9.80 - ETA: 22s - loss: 9.62 - ETA: 16s - loss: 9.38 - ETA: 11s - loss: 9.14 - ETA: 6s - loss: 8.9222 - ETA: 1s - loss: 8.727 - 44s 145ms/step - loss: 8.6491
Epoch 2/20
300/300 [==============================] - ETA: 36s - loss: 6.67 - ETA: 31s - loss: 6.71 - ETA: 25s - loss: 6.69 - ETA: 21s - loss: 6.69 - ETA: 16s - loss: 6.70 - ETA: 12s - loss: 6.70 - ETA: 8s - loss: 6.7056 - ETA: 5s - loss: 6.691 - ETA: 1s - loss: 6.677 - 35s 116ms/step - loss: 6.6754
Epoch 3/20
300/300 [==============================] - ETA: 29s - loss: 6.37 - ETA: 26s - loss: 6.43 - ETA: 22s - loss: 6.47 - ETA: 19s - loss: 6.50 - ETA: 15s - loss: 6.47 - ETA: 11s - loss: 6.48 - ETA: 8s - loss: 6.4980 - ETA: 4s - loss: 6.504 - ETA

300/300 [==============================] - ETA: 29s - loss: 4.87 - ETA: 25s - loss: 4.89 - ETA: 22s - loss: 4.89 - ETA: 18s - loss: 4.85 - ETA: 15s - loss: 4.83 - ETA: 12s - loss: 4.82 - ETA: 8s - loss: 4.8330 - ETA: 4s - loss: 4.813 - ETA: 1s - loss: 4.804 - 34s 112ms/step - loss: 4.8095
Epoch 7/20
300/300 [==============================] - ETA: 30s - loss: 4.60 - ETA: 26s - loss: 4.68 - ETA: 23s - loss: 4.62 - ETA: 19s - loss: 4.63 - ETA: 15s - loss: 4.64 - ETA: 12s - loss: 4.64 - ETA: 8s - loss: 4.6599 - ETA: 5s - loss: 4.670 - ETA: 1s - loss: 4.671 - 34s 114ms/step - loss: 4.6738
Epoch 8/20
300/300 [==============================] - ETA: 29s - loss: 4.57 - ETA: 26s - loss: 4.52 - ETA: 22s - loss: 4.51 - ETA: 19s - loss: 4.52 - ETA: 15s - loss: 4.54 - ETA: 12s - loss: 4.54 - ETA: 8s - loss: 4.5500 - ETA: 4s - loss: 4.548 - ETA: 1s - loss: 4.545 - 34s 112ms/step - loss: 4.5433
Epoch 9/20
300/300 [==============================] - ETA: 29s - loss: 4.46 - ETA: 26s - loss: 4.44 - ETA: 2

300/300 [==============================] - ETA: 29s - loss: 2.20 - ETA: 26s - loss: 2.16 - ETA: 22s - loss: 2.18 - ETA: 19s - loss: 2.20 - ETA: 15s - loss: 2.21 - ETA: 12s - loss: 2.23 - ETA: 8s - loss: 2.2547 - ETA: 4s - loss: 2.254 - ETA: 1s - loss: 2.252 - 34s 112ms/step - loss: 2.2599
Epoch 14/20
300/300 [==============================] - ETA: 30s - loss: 2.27 - ETA: 26s - loss: 2.21 - ETA: 22s - loss: 2.22 - ETA: 19s - loss: 2.19 - ETA: 15s - loss: 2.18 - ETA: 12s - loss: 2.17 - ETA: 8s - loss: 2.1831 - ETA: 4s - loss: 2.173 - ETA: 1s - loss: 2.169 - 34s 112ms/step - loss: 2.1688
Epoch 15/20
300/300 [==============================] - ETA: 30s - loss: 2.04 - ETA: 26s - loss: 2.08 - ETA: 22s - loss: 2.08 - ETA: 19s - loss: 2.07 - ETA: 15s - loss: 2.06 - ETA: 12s - loss: 2.07 - ETA: 8s - loss: 2.0881 - ETA: 4s - loss: 2.094 - ETA: 1s - loss: 2.087 - 34s 112ms/step - loss: 2.0858
Epoch 16/20
300/300 [==============================] - ETA: 29s - loss: 1.99 - ETA: 26s - loss: 2.00 - ETA

300/300 [==============================] - ETA: 30s - loss: 0.93 - ETA: 26s - loss: 0.93 - ETA: 23s - loss: 0.94 - ETA: 19s - loss: 0.98 - ETA: 15s - loss: 0.98 - ETA: 12s - loss: 0.99 - ETA: 8s - loss: 0.9754 - ETA: 4s - loss: 0.990 - ETA: 1s - loss: 1.005 - 34s 112ms/step - loss: 1.0021
Currently--> 1200:1500
Epoch 1/20
300/300 [==============================] - ETA: 29s - loss: 2.44 - ETA: 26s - loss: 2.50 - ETA: 22s - loss: 2.30 - ETA: 20s - loss: 2.29 - ETA: 17s - loss: 2.23 - ETA: 13s - loss: 2.24 - ETA: 9s - loss: 2.2149 - ETA: 5s - loss: 2.200 - ETA: 1s - loss: 2.183 - 39s 130ms/step - loss: 2.1629
Epoch 2/20
300/300 [==============================] - ETA: 34s - loss: 1.89 - ETA: 29s - loss: 1.90 - ETA: 25s - loss: 1.85 - ETA: 20s - loss: 1.86 - ETA: 16s - loss: 1.84 - ETA: 12s - loss: 1.82 - ETA: 8s - loss: 1.8516 - ETA: 5s - loss: 1.843 - ETA: 1s - loss: 1.861 - 35s 116ms/step - loss: 1.8697
Epoch 3/20
300/300 [==============================] - ETA: 30s - loss: 1.51 - ETA: 26

300/300 [==============================] - ETA: 29s - loss: 1.05 - ETA: 26s - loss: 0.98 - ETA: 22s - loss: 1.03 - ETA: 19s - loss: 1.03 - ETA: 15s - loss: 1.01 - ETA: 11s - loss: 1.01 - ETA: 8s - loss: 1.0355 - ETA: 4s - loss: 1.034 - ETA: 1s - loss: 1.025 - 33s 111ms/step - loss: 1.0299
Epoch 8/20
300/300 [==============================] - ETA: 29s - loss: 0.87 - ETA: 26s - loss: 0.91 - ETA: 22s - loss: 0.93 - ETA: 19s - loss: 0.94 - ETA: 15s - loss: 0.94 - ETA: 12s - loss: 0.96 - ETA: 8s - loss: 0.9540 - ETA: 4s - loss: 0.954 - ETA: 1s - loss: 0.963 - 33s 112ms/step - loss: 0.9597
Epoch 9/20
300/300 [==============================] - ETA: 29s - loss: 0.75 - ETA: 26s - loss: 0.79 - ETA: 22s - loss: 0.82 - ETA: 19s - loss: 0.85 - ETA: 15s - loss: 0.87 - ETA: 12s - loss: 0.88 - ETA: 8s - loss: 0.8942 - ETA: 4s - loss: 0.883 - ETA: 1s - loss: 0.888 - 34s 112ms/step - loss: 0.8940
Epoch 10/20
300/300 [==============================] - ETA: 29s - loss: 0.73 - ETA: 26s - loss: 0.78 - ETA: 

...

300/300 [==============================] - ETA: 29s - loss: 0.06 - ETA: 25s - loss: 0.06 - ETA: 22s - loss: 0.05 - ETA: 18s - loss: 0.05 - ETA: 15s - loss: 0.06 - ETA: 11s - loss: 0.06 - ETA: 8s - loss: 0.0746 - ETA: 4s - loss: 0.072 - ETA: 1s - loss: 0.073 - 33s 110ms/step - loss: 0.0736
Epoch 15/20
300/300 [==============================] - ETA: 29s - loss: 0.05 - ETA: 26s - loss: 0.06 - ETA: 22s - loss: 0.06 - ETA: 18s - loss: 0.06 - ETA: 15s - loss: 0.06 - ETA: 11s - loss: 0.06 - ETA: 8s - loss: 0.0669 - ETA: 4s - loss: 0.064 - ETA: 1s - loss: 0.065 - 33s 110ms/step - loss: 0.0643
Epoch 16/20
300/300 [==============================] - ETA: 29s - loss: 0.05 - ETA: 25s - loss: 0.06 - ETA: 22s - loss: 0.06 - ETA: 18s - loss: 0.05 - ETA: 15s - loss: 0.05 - ETA: 11s - loss: 0.05 - ETA: 8s - loss: 0.0511 - ETA: 4s - loss: 0.053 - ETA: 1s - loss: 0.056 - 33s 109ms/step - loss: 0.0571
Epoch 17/20
300/300 [==============================] - ETA: 28s - loss: 0.05 - ETA: 26s - loss: 0.05 - ETA

Epoch 1/20
300/300 [==============================] - ETA: 29s - loss: 0.59 - ETA: 25s - loss: 0.53 - ETA: 22s - loss: 0.53 - ETA: 18s - loss: 0.54 - ETA: 15s - loss: 0.55 - ETA: 11s - loss: 0.55 - ETA: 8s - loss: 0.5324 - ETA: 4s - loss: 0.532 - ETA: 1s - loss: 0.518 - 33s 111ms/step - loss: 0.5245
Epoch 2/20
300/300 [==============================] - ETA: 29s - loss: 0.29 - ETA: 26s - loss: 0.38 - ETA: 22s - loss: 0.38 - ETA: 19s - loss: 0.38 - ETA: 15s - loss: 0.40 - ETA: 11s - loss: 0.43 - ETA: 8s - loss: 0.4166 - ETA: 4s - loss: 0.419 - ETA: 1s - loss: 0.408 - 33s 111ms/step - loss: 0.4074
Epoch 3/20
300/300 [==============================] - ETA: 29s - loss: 0.34 - ETA: 25s - loss: 0.30 - ETA: 22s - loss: 0.31 - ETA: 19s - loss: 0.33 - ETA: 15s - loss: 0.32 - ETA: 12s - loss: 0.31 - ETA: 8s - loss: 0.3252 - ETA: 4s - loss: 0.331 - ETA: 1s - loss: 0.344 - 33s 111ms/step - loss: 0.3420
Epoch 4/20
300/300 [==============================] - ETA: 29s - loss: 0.32 - ETA: 25s - loss: 0.

300/300 [==============================] - ETA: 29s - loss: 0.15 - ETA: 25s - loss: 0.14 - ETA: 22s - loss: 0.14 - ETA: 18s - loss: 0.13 - ETA: 15s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 8s - loss: 0.1230 - ETA: 4s - loss: 0.119 - ETA: 1s - loss: 0.121 - 33s 110ms/step - loss: 0.1221
Epoch 9/20
300/300 [==============================] - ETA: 29s - loss: 0.10 - ETA: 25s - loss: 0.11 - ETA: 22s - loss: 0.11 - ETA: 18s - loss: 0.10 - ETA: 15s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 8s - loss: 0.1062 - ETA: 4s - loss: 0.104 - ETA: 1s - loss: 0.104 - 33s 110ms/step - loss: 0.1037
Epoch 10/20
300/300 [==============================] - ETA: 28s - loss: 0.07 - ETA: 25s - loss: 0.09 - ETA: 22s - loss: 0.09 - ETA: 18s - loss: 0.10 - ETA: 15s - loss: 0.10 - ETA: 11s - loss: 0.09 - ETA: 8s - loss: 0.0949 - ETA: 4s - loss: 0.090 - ETA: 1s - loss: 0.090 - 33s 109ms/step - loss: 0.0898
Epoch 11/20
300/300 [==============================] - ETA: 29s - loss: 0.07 - ETA: 25s - loss: 0.07 - ETA:

300/300 [==============================] - ETA: 28s - loss: 0.02 - ETA: 25s - loss: 0.02 - ETA: 22s - loss: 0.04 - ETA: 18s - loss: 0.05 - ETA: 15s - loss: 0.05 - ETA: 11s - loss: 0.05 - ETA: 8s - loss: 0.0604 - ETA: 4s - loss: 0.060 - ETA: 1s - loss: 0.059 - 33s 110ms/step - loss: 0.0605
Epoch 16/20
300/300 [==============================] - ETA: 29s - loss: 0.09 - ETA: 25s - loss: 0.07 - ETA: 22s - loss: 0.06 - ETA: 18s - loss: 0.05 - ETA: 15s - loss: 0.05 - ETA: 11s - loss: 0.05 - ETA: 8s - loss: 0.0545 - ETA: 4s - loss: 0.054 - ETA: 1s - loss: 0.053 - 33s 110ms/step - loss: 0.0527
Epoch 17/20
300/300 [==============================] - ETA: 29s - loss: 0.06 - ETA: 25s - loss: 0.04 - ETA: 22s - loss: 0.04 - ETA: 18s - loss: 0.04 - ETA: 15s - loss: 0.04 - ETA: 11s - loss: 0.04 - ETA: 8s - loss: 0.0453 - ETA: 4s - loss: 0.048 - ETA: 1s - loss: 0.047 - 33s 110ms/step - loss: 0.0467
Epoch 18/20
300/300 [==============================] - ETA: 29s - loss: 0.03 - ETA: 25s - loss: 0.03 - ETA

300/300 [==============================] - ETA: 28s - loss: 0.36 - ETA: 25s - loss: 0.40 - ETA: 22s - loss: 0.36 - ETA: 19s - loss: 0.36 - ETA: 15s - loss: 0.38 - ETA: 11s - loss: 0.38 - ETA: 8s - loss: 0.3797 - ETA: 4s - loss: 0.377 - ETA: 1s - loss: 0.379 - 33s 111ms/step - loss: 0.3740
Epoch 3/20
300/300 [==============================] - ETA: 30s - loss: 0.38 - ETA: 26s - loss: 0.36 - ETA: 22s - loss: 0.32 - ETA: 18s - loss: 0.30 - ETA: 15s - loss: 0.28 - ETA: 11s - loss: 0.29 - ETA: 8s - loss: 0.3144 - ETA: 4s - loss: 0.314 - ETA: 1s - loss: 0.313 - 33s 111ms/step - loss: 0.3107
Epoch 4/20
300/300 [==============================] - ETA: 28s - loss: 0.23 - ETA: 25s - loss: 0.19 - ETA: 22s - loss: 0.23 - ETA: 18s - loss: 0.24 - ETA: 15s - loss: 0.25 - ETA: 11s - loss: 0.25 - ETA: 8s - loss: 0.2494 - ETA: 4s - loss: 0.256 - ETA: 1s - loss: 0.254 - 33s 110ms/step - loss: 0.2581
Epoch 5/20
300/300 [==============================] - ETA: 29s - loss: 0.23 - ETA: 25s - loss: 0.25 - ETA: 2

In [66]:
model.save("chat_botV1.h5")

In [67]:
# testing = load_model("chat_botV1.h5")

In [68]:
encoder_states = state_h, state_c

In [69]:
def make_inference_model():
    
    encoder_model = Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=(output_dim,))
    decoder_state_input_c = Input(shape=(output_dim,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_LSTM(decoder_embedding, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_ouputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                         [decoder_outputs] + decoder_states)
    
    return encoder_model, decoder_model

In [70]:
def format_questions(sentence):
    words = sentence.lower().split()
    sentence_words = list()
    for word in words:
        sentence_words.append(tokenizer.word_index[word])
    padded_input = pad_sequences([sentence_words], max_seq_len)
    
    return padded_input

In [71]:
print(format_questions("What can I do about cardiac pain"))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  24  18  10  28  74 163  49]]


In [ ]:
enc_model, dec_model = make_inference_model()

for i in range (10):
    state_values = enc_model.predict(format_questions(input('Enter question: ')))
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq] + state_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                decoded_translation += " {}".format(word)
                sampled_word = word
            
            if sampled_word == 'end' or len(decoded_translation.split()) > max_seq_lenAns:
                stop_condition = True
            
            empty_target_seq = np.zeros ((1, 1))
            empty_target_seq[0, 0] = sampled_word_index
            states_values = [h, c]
            
    print(decoded_translation)

Enter question: What could be a sign of cardiac arrest
 for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for has for
Enter question: What is cardiovascular disease
 for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for all for
Enter question: how do i lose weight
 for sleep for sleep for sleep for sleep for sleep 